# Main experiment (TF2.10)

This notebook produces the main experimental pipeline:
- Load CIFAR-10 and adversarial samples produced in `01` (Torch AutoAttack) and `03` (TF2 + ART APGD).
- Cache per-model predictions and compute ensemble entropy.
- Sweep entropy thresholds (core vs out-of-core) and report accuracy/coverage.
- Specialize **second-generation $\mathcal{U}^{(2)^\prime}$** on the out-of-core region and evaluate the full IMM pipeline.



In [ ]:
from __future__ import annotations

import os, sys, glob, yaml, csv, time
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Tuple, Any, TypedDict

import numpy as np
import tensorflow as tf
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('./source')

import source.cache_store as cache_store
import source.custom_specialization as custom_specialization 
import source.utils as utils

In [ ]:
def load_yaml(path: str):
    with open(path, "r") as f:
        return yaml.safe_load(f)

PATHS = load_yaml("./configs/paths.yaml")
EXP   = load_yaml("./configs/exp.yaml")

data_root    = Path(PATHS["data_root"])
results_root = Path(PATHS["results_root"])
tf_model_dir = Path(PATHS["tf_model_dir"])
SPs_model_dir = Path(PATHS["SPs_model_dir"])
autoattack_out = Path(PATHS["autoattack_out"])
apgd_out     = Path(PATHS["apgd_out"])
cache_root   = Path(PATHS["cache_root"])

seed = int(EXP["seed"])
np.random.seed(seed)
tf.random.set_seed(seed)

# Ensure dirs
results_root.mkdir(parents=True, exist_ok=True)
cache_root.mkdir(parents=True, exist_ok=True)

print("tf_model_dir:", tf_model_dir)
print("autoattack_out:", autoattack_out)
print("apgd_out:", apgd_out)
print("cache_root:", cache_root)


## Load CIFAR-10 (Keras)

In [ ]:
(x_all, y_all), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_all  = y_all.reshape(-1).astype(np.int64)
y_test = y_test.reshape(-1).astype(np.int64)

x_train, x_val, y_train, y_val = train_test_split(
    x_all, y_all, test_size=0.2, random_state=seed, stratify=y_all
)

x_train = x_train.astype(np.float32)/255.0
x_val   = x_val.astype(np.float32)/255.0
x_test  = x_test.astype(np.float32)/255.0

y_train = y_train.astype(np.int64)
y_val   = y_val.astype(np.int64)


## Load adversarial samples from 01 (Torch AutoAttack) and 03 (ART APGD)

In [ ]:
samples: Dict[str, np.ndarray] = {}
y_true: Dict[str, np.ndarray] = {}

# Original
samples["original_train"] = x_train.copy()
samples["original_val"]   = x_val.copy()
samples["original_test"]  = x_test.copy()
y_true["original_train"] = y_train.copy()
y_true["original_val"]   = y_val.copy()
y_true["original_test"]  = y_test.copy()

# Torch AutoAttack outputs (saved as .npy in NCHW; convert to NHWC)
aa_path = autoattack_out / "x_adv_test_std_l2_eps5.npy"  # adjust if needed
x_adv = np.load(aa_path).astype(np.float32)

if x_adv.ndim == 4:
    if x_adv.shape[-1] == 3 and x_adv.shape[1] != 3:
        # already NHWC: (N, H, W, C)
        pass
    elif x_adv.shape[1] == 3 and x_adv.shape[-1] != 3:
        # NCHW: (N, C, H, W) -> NHWC
        x_adv = np.transpose(x_adv, (0, 2, 3, 1))
samples["AA_std_test"] = x_adv
y_true["AA_std_test"] = y_test.copy()

# ART APGD outputs -> key = "{tag}_{split}_{weak/strong}"
for p in sorted(apgd_out.glob("*.npz")):
    d = np.load(p)

    split = str(d["split"]) if "split" in d else "unknown"
    tag   = str(d["tag"])   if "tag"   in d else "unknown"
    it    = int(d["max_iter"]) if "max_iter" in d else None

    strength = "weak" if it == 2 else ("strong" if it == 10 else f"it{it}")
    sample_name = f"{tag}_{strength}_{split}"

    samples[sample_name] = d["x_adv"].astype(np.float32)
    y_true[sample_name]  = d["y"].astype(np.int64).reshape(-1)
    
samples["U0_union_train"] = np.concatenate(
    [samples["original_train"], samples["U0_weak_train"], samples["U0_strong_train"]],
    axis=0
).astype(np.float32)
y_true["U0_union_train"] = np.concatenate(
    [y_true["original_train"], y_true["U0_weak_train"], y_true["U0_strong_train"]],
    axis=0
).astype(np.int64)

samples["U0_union_val"] = np.concatenate(
    [samples["original_val"], samples["U0_weak_val"], samples["U0_strong_val"]],
    axis=0
).astype(np.float32)
y_true["U0_union_val"] = np.concatenate(
    [y_true["original_val"], y_true["U0_weak_val"], y_true["U0_strong_val"]],
    axis=0
).astype(np.int64)

samples["U0_union_test"] = np.concatenate(
    [samples["original_test"], samples["U0_weak_test"], samples["U0_strong_test"]],
    axis=0
).astype(np.float32)
y_true["U0_union_test"] = np.concatenate(
    [y_true["original_test"], y_true["U0_weak_test"], y_true["U0_strong_test"]],
    axis=0
).astype(np.int64)

## Load models and build first-layer committee (U0 ∪ U1)

In [ ]:
def load_models_from_dir(model_dir: Path, pattern: str="*.keras", exclude_substr: str="original") -> Dict[str, tf.keras.Model]:
    md = {}
    for p in sorted(model_dir.glob(pattern)):
        if exclude_substr and exclude_substr in p.name:
            continue
        md[p.name] = load_model(str(p))
    return md

# U0 baseline models
u0 = load_models_from_dir(tf_model_dir, pattern="*.keras")
sps = load_models_from_dir(SPs_model_dir, pattern="*.keras")
u1 = {}
u1.update(u0)
u1.update(sps)


In [ ]:
utils.clear_folder(str(cache_root), dry_run=False)
caching = cache_store.ResultStore(root=str(cache_root))

# committees
u0_keys = sorted(u0.keys())
u1_keys = sorted(u1.keys())
sps_keys = sorted(sps.keys())

# predict-cache: U1 (== First_layer) only (contains U0 + SPs)
participating_models = u1

for sample_name, X in samples.items():
    Y = y_true[sample_name]
    print(f"{len(participating_models)} models caching preds on {sample_name}...", end="---")

    for model_name, model in participating_models.items():
        pred = model.predict(X, verbose=0)
        caching.set_pred(model_name, sample_name, pred)
    print("Done.")

    # entropy for U0
    print(f"caching entropy (U0) on {sample_name}...", end="---")
    arr_u0 = np.array([caching.get_pred(mn, sample_name) for mn in u0_keys])  # (M,N,C)
    ent_u0 = np.array([utils.cross_entropy(arr_u0[:, i, :]) for i in range(arr_u0.shape[1])])
    caching.set_entropy(u0_keys, sample_name, ent_u0)
    print("Done.")

    # entropy for U1 (= U0 + SPs)
    print(f"caching entropy (U1) on {sample_name}...", end="---")
    arr_u1 = np.array([caching.get_pred(mn, sample_name) for mn in u1_keys])
    ent_u1 = np.array([utils.cross_entropy(arr_u1[:, i, :]) for i in range(arr_u1.shape[1])])
    caching.set_entropy(u1_keys, sample_name, ent_u1)
    print("Done.\n")

In [ ]:
class ProbAverageEnsembleFromProbModels(tf.keras.Model):
    def __init__(self, prob_models : dict, eps=1e-12, cache_root : str = './data/cache/'):
        super().__init__()
        self.prob_models = prob_models
        self.eps = eps
        self.caching = cache_store.ResultStore(root = cache_root)
        
    def probs(self, x, sample_name : str = ''):
        '''
        return average probability (average softmax value)
        '''
        if sample_name:
            self.caching_pred(x, sample_name)
            preds = [self.caching.get_pred(model_name, sample_name) for model_name in sorted(self.prob_models)]
        else:
            preds = [self.prob_models[k].predict(x,verbose = 0, batch_size = 64) for k in sorted(self.prob_models)]
        
        return np.mean(preds, axis=0)
    def core_preds(self, x, sample_name : str = '', ent_th : float = 0.1):
        '''
        return average probability (average softmax value)
        '''
        if sample_name:
            self.caching_pred(x, sample_name)
            preds = [self.caching.get_pred(model_name, sample_name) for model_name in sorted(self.prob_models)]
        else:
            preds = [self.prob_models[k].predict(x,verbose = 0, batch_size = 64) for k in sorted(self.prob_models)]
        preds = np.array(preds)
        
        if sample_name:
            self.caching_ent(sample_name)
            ent = self.caching.get_entropy(sorted(self.prob_models), sample_name)
        else:
            ent = []
            for i in range(preds.shape[1]):
                ent.append(self.cross_entropy(preds[:,i,: ]))
            ent = np.array(ent)
        
        core = ent<ent_th
        
        preds = preds[:, core, :]
        core_preds = np.mean(preds, axis=0)
        
        return core_preds, core, ent
        
            
    
    
    def caching_pred(self, x, sample_name):
        for model_name, model in self.prob_models.items():
            key = (self.caching.int_from_model_path(model_name), sample_name)
            fp = self.caching._preds_idx.get(key)

            if fp and Path(fp).exists():
                # Already cached and file exists: nothing to do.
#                 print(f'{model_name} already made predictions on {sample_name} and it was cached.')
                
                continue

            elif fp and not Path(fp).exists():
                # Index says there is a file, but it doesn't exist anymore.
                # Recompute and overwrite.
#                 print(f"Index says there is a file, but it doesn't exist anymore. {model_name}  on {sample_name}. Recompute and Overwrite.")

                pred = model.predict(x, verbose=0)
                self.caching.set_pred(model_name, sample_name, pred)

            else:
                # No index entry; check if file exists under the naming convention.
#                 print(f'{model_name} make predictions on {sample_name} and it is cached.')
                outdir = self.caching.root / "preds" / sample_name
                fp = outdir / f"{self.caching._stem(model_name)}.npy"
                if Path(fp).exists():
#                     print('File exists but is not in the index: add it.')
                    # File exists but is not in the index: add it.
                    pred = np.load(fp)
                    key = (self.caching.int_from_model_path(model_name), sample_name)
                    self.caching._preds_idx[key] = str(fp)
                    self.caching._dump_idx(self.caching._preds_idx_path, self.caching._preds_idx)
                else:
#                     print('processing..', end = ' ')
                    # Compute and cache.
                    pred = model.predict(x, verbose=0)
                    self.caching.set_pred(model_name, sample_name, pred)
#                     print('Done.')
                    
    def caching_ent(self, sample_name):
        # Get predictions for each model.
        sig = self.caching._committee_sig(list(self.prob_models.keys()))
        key = ("entropy", sig, sample_name)
        fp = self.caching._metrics_idx.get(key)

        # Caching entropy
        if fp and Path(fp).exists():
#             print('sample', sample_name, 'entropy cached already')
            # Already cached and file exists: fine.
            pass

        elif fp and not Path(fp).exists():
            # Index points to missing file: recompute and overwrite.
            arrays = np.array([self.caching.get_pred(model_name, sample_name) for model_name in sorted(self.prob_models.keys())])

            ent = []
            for i in range(arrays.shape[1]):
                ent.append(self.cross_entropy(arrays[:,i,: ]))
            ent = np.array(ent)
            self.caching.set_entropy(sorted(self.prob_models.keys()), sample_name, ent)

        else:
            # No index entry; check if entropy file exists under naming convention.
            outdir = self.caching.root / "metrics" / sample_name
            fp = outdir / f"entropy__{sig}.npy"
            if Path(fp).exists():
                key = ("entropy", sig, sample_name)
                self.caching._metrics_idx[key] = str(fp)
                self.caching._dump_idx(self.caching._metrics_idx_path, self.caching._metrics_idx)
            else:
                # Compute and cache.
                arrays = np.array([self.caching.get_pred(model_name, sample_name) for model_name in sorted(self.prob_models.keys())])

                ent = []
                for i in range(arrays.shape[1]):
                    ent.append(self.cross_entropy(arrays[:,i,: ]))
                ent = np.array(ent)
                self.caching.set_entropy(sorted(self.prob_models.keys()), sample_name, ent)
    
    def cross_entropy(self, arrays):
        n = len(arrays)
        if n == 0:
            raise ValueError("At least one array is required.")

        # Stack to shape (m, n)
        cols = [np.asarray(a).reshape(-1) for a in arrays]
        m = cols[0].shape[0]
        if any(c.shape[0] != m for c in cols):
            raise ValueError("All arrays must have the same first dimension m.")
        A = np.column_stack(cols)  # shape (m, n)

        log_base = m  
        L = np.log(A + 1e-12) / np.log(log_base)

        M = A.T @ L  # shape (n, n), M[k, l] = sum_i a_{i,k} * log(a_{i,l})
        total = -np.sum(M) # sum over all pair
        
        return float(total)/(n**2)


In [ ]:
TARGET_ACC = 0.95
STEP = 0.01
MAX_TH = 2.0
th_grid = np.arange(STEP, MAX_TH + 1e-9, STEP)

def find_th_for_core_acc(ens, sample_name, X, y, th_grid):
    for th in th_grid:
        core_preds, core, _ = ens.core_preds(X, sample_name=sample_name, ent_th=float(th))
        cnt = int(core.sum())
        if cnt < 10:
            continue
        core_acc = float(np.mean(np.argmax(core_preds, axis=-1) == y[core]))
        if core_acc >= TARGET_ACC:
            core_cov = float(core.mean())
            return float(th), core_acc, core_cov, cnt
    return None

ens_u1 = ProbAverageEnsembleFromProbModels(u1, cache_root=str(cache_root))

ent_th_dict = {}

for sample_name, X in samples.items():
    if not sample_name.endswith("_val"):
        continue

    y = y_true[sample_name]
    head = "_".join(sample_name.split("_")[:-1])  # drop "_val"

    out = find_th_for_core_acc(ens_u1, sample_name, X, y, th_grid)
    if out is None:
        ent_th_dict[head] = None
        print(head, None, None, None)
    else:
        th, core_acc, core_cov, core_cnt = out
        ent_th_dict[head] = th

In [ ]:
def eval_entropy_fuzzy(
    participating_models: dict[str, tf.keras.Model],
    caching: cache_store.CacheStore,
    samples: dict[str, np.ndarray],
    y_true_dict: dict[str, np.ndarray],
    ent_th: float,
    verbose: bool = True,
):
    sample_names = list(samples.keys())
    results = {}

    keys_sorted = sorted(participating_models.keys())

    for sample_name in sample_names:
        y_true = y_true_dict[sample_name].reshape(-1)

        ent = caching.get_entropy(keys_sorted, sample_name)
        low_idx = ent <= ent_th
        high_idx = ~low_idx

        low_cnt = int(np.sum(low_idx))
        high_cnt = int(np.sum(high_idx))

        # Per-model stats
        per_model_rows = []
        for model_name in keys_sorted:
            pred = caching.get_pred(model_name, sample_name)
            pred_label = np.argmax(pred, axis=1)

            global_acc = float(np.mean(pred_label == y_true))
            low_acc = float(np.mean(pred_label[low_idx] == y_true[low_idx])) if low_cnt > 0 else 0.0
            high_acc = float(np.mean(pred_label[high_idx] == y_true[high_idx])) if high_cnt > 0 else 0.0

            per_model_rows.append({
                "model": str(Path(model_name).stem),
                "global_acc": global_acc,
                "low_ent_acc": low_acc,
                "high_ent_acc": high_acc,
            })

        per_model_df = pd.DataFrame(per_model_rows)

        # Ensemble stats
        preds_all = np.array([caching.get_pred(mn, sample_name) for mn in keys_sorted])  # (M,N,C)
        ens_raw = np.mean(preds_all, axis=0)  # (N,C)
        overall_acc = float(np.mean(np.argmax(ens_raw, axis=1) == y_true))

        # Low entropy: zero out high region before averaging
        preds_low = preds_all.copy()
        preds_low[:, ~low_idx, :] = 0
        ens_low = np.mean(preds_low, axis=0)

        # High entropy: zero out low region before averaging
        preds_high = preds_all.copy()
        preds_high[:, low_idx, :] = 0
        ens_high = np.mean(preds_high, axis=0)

        low_acc_ens = float(np.mean(np.argmax(ens_low[low_idx], axis=1) == y_true[low_idx])) if low_cnt > 0 else 0.0
        high_acc_ens = float(np.mean(np.argmax(ens_high[high_idx], axis=1) == y_true[high_idx])) if high_cnt > 0 else 0.0

        ensemble_summary = {
            "overall_acc": overall_acc,
            "low_entropy_count": low_cnt,
            "low_entropy_acc": low_acc_ens,
            "high_entropy_count": high_cnt,
            "high_entropy_acc": high_acc_ens,
        }

        results[sample_name] = {
            "entropy_threshold": float(ent_th),
            "per_model_df": per_model_df,
            "ensemble": ensemble_summary,
        }

        if verbose:
            print("\n" + "=" * 70)
            print(f"Sample: {sample_name}")
            print("-" * 70)
            print(f"Entropy threshold: {ent_th:.6f}")

            print("\nPer-model performance:")
            print(per_model_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

            print("\nEnsemble performance:")
            print(
                f"  Overall accuracy:     {overall_acc:.4f}\n"
                f"  Low-entropy count:    {low_cnt}\n"
                f"  Low-entropy accuracy: {low_acc_ens:.4f}\n"
                f"  High-entropy count:   {high_cnt}\n"
                f"  High-entropy accuracy:{high_acc_ens:.4f}\n"
            )

    return results


def plot_ent_th_sweep(
    df,
    overall_acc,
    sample_name="AA_RBST_test",
    outdir=None,
    vline_ent_ths: list[float] = [0.0414, 1.0],
    show_legend: bool = False,
    legend_outside: bool = True,
):
    assert len(vline_ent_ths) == 2, "vline_ent_ths must have two thresholds (low and high)."

    df = df.copy()
    df["ent_th"] = df["ent_th"].astype(float)
    df = df.sort_values("ent_th").reset_index(drop=True)

    x = df["ent_th"].to_numpy()
    core_cnt = df["core count"].to_numpy(dtype=float)
    high_cnt = df["high count"].to_numpy(dtype=float)
    acc1 = df["acc1"].to_numpy(dtype=float)
    acc2 = df["acc2"].to_numpy(dtype=float)

    total_n = core_cnt + high_cnt
    N = float(total_n[0])

    def _save(fig, name):
        if outdir is None:
            return
        os.makedirs(outdir, exist_ok=True)
        fig.savefig(os.path.join(outdir, name), dpi=300)

    def nearest_idx(arr, val):
        arr = np.asarray(arr, dtype=float)
        return int(np.argmin(np.abs(arr - float(val))))

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    ax1.plot(x, core_cnt / N, label="core coverage", linestyle="--", color="orange")
    ax1.plot(x, high_cnt / N, label="high coverage", linestyle="-.", color="blue")

    ax2.plot(x, acc1, linestyle="-", label="acc (core)", color="black")
    ax2.plot(x, acc2, linestyle="-", label="acc (high)", color="red")

    ax2.axhline(float(overall_acc), linestyle=":")
    ax2.annotate(
        f"overall acc\n{overall_acc:.4f}",
        (x[-1], float(overall_acc)),
        textcoords="offset points",
        xytext=(18, 8),
        ha="left",
        va="bottom",
        fontsize=9,
    )

    ax1.set_xlabel("Entropy Threshold")
    ax1.set_ylabel("Coverage")
    ax2.set_ylabel("Accuracy")
    ax1.set_ylim(0, 1.1)
    ax2.set_ylim(0, 1.1)

    ax1.grid(False)
    ax2.grid(False)
    plt.yticks([])

    core_loc = nearest_idx(x, vline_ent_ths[0])

    ax1.axvline(x[core_loc], linestyle="--", alpha=0.7)
    ax1.annotate(
        f"{vline_ent_ths[0]:.1f}",
        (x[core_loc], 0),
        textcoords="offset points",
        xytext=(6, -6),
        ha="left",
        va="top",
        fontsize=11,
    )

    ax1.plot([x[core_loc]], [core_cnt[core_loc] / N], marker="o", linestyle="None", color="black")
    ax1.annotate(
        f"{core_cnt[core_loc] / N:.4f}",
        (x[core_loc], core_cnt[core_loc] / N),
        textcoords="offset points",
        xytext=(-6, 4),
        ha="right",
        va="bottom",
        fontsize=11,
    )

    ax1.set_xscale("log")

    fig.subplots_adjust(right=0.85, top=0.82)

    if show_legend:
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        lines = lines1 + lines2
        labels = labels1 + labels2

        if legend_outside:
            ax1.legend(
                lines,
                labels,
                loc="lower center",
                bbox_to_anchor=(0.5, 1.02),
                ncol=len(labels),
                frameon=True,
                borderaxespad=0.0,
                handlelength=2.0,
                columnspacing=1.2,
            )
        else:
            ax1.legend(lines, labels, loc="best")

    _save(fig, f"{sample_name}_counts_and_acc_dualaxis.png")
    plt.show()

In [ ]:
# adjust this for each figures.
high_ents = np.linspace(1.05, 0.1, 8)
low_ents = np.linspace(0.15, np.log(1.1)/np.log(10), 6)
very_low_ents = np.linspace(np.log(1.1)/np.log(10), 0.001, 8)

base_ent_ths = np.unique(np.concatenate([high_ents, low_ents, very_low_ents]))
base_ent_ths = np.sort(base_ent_ths)[::-1]

legend_outside = False
show_legend = False
per_sample_ent_th_results = {}

for sample in sorted(samples):
    if not sample.endswith("_val"):
        continue

    head = "_".join(sample.split("_")[:-1])  # drop "_val"
    vth = ent_th_dict.get(head, None)

    # add per-sample vth into sweep thresholds
    if vth is None:
        ent_ths = base_ent_ths
        low_and_high = [0.1, 1.0]
    else:
        ent_ths = np.unique(np.concatenate([base_ent_ths, np.array([vth], dtype=float)]))
        ent_ths = np.sort(ent_ths)[::-1]
        low_and_high = [float(round(vth, 4)), 1.0]

    rows = []
    for ent_th in ent_ths:
        ent_th = float(round(float(ent_th), 4))
        result = eval_entropy_fuzzy(
            participating_models=u1,
            caching=caching,
            samples={sample: samples[sample]},
            y_true_dict={sample: y_true[sample]},
            ent_th=ent_th,
            verbose=False,
        )
        per_sample_ent_th_results[(sample, ent_th)] = result
        ens = result[sample]["ensemble"]
        rows.append({
            "ent_th": ent_th,
            "core count": ens["low_entropy_count"],
            "acc1": ens["low_entropy_acc"],
            "high count": ens["high_entropy_count"],
            "acc2": ens["high_entropy_acc"],
        })

    df_sweep = pd.DataFrame(rows)
    overall_acc = per_sample_ent_th_results[(sample, float(round(float(ent_ths[0]), 4)))][sample]["ensemble"]["overall_acc"]

    outdir = f'./analysis/figures/{sample}_u1_val'
    plot_ent_th_sweep(
        df_sweep,
        overall_acc=overall_acc,
        sample_name=sample,
        outdir=outdir,
        vline_ent_ths=low_and_high,
        show_legend=show_legend,
        legend_outside=legend_outside
    )

In [ ]:
# Single model entropy stats (clean vs adv) aggregated by family (resnet/vgg)
truth = y_true["original_test"].reshape(-1)

ADV_DIR = apgd_out  # use your configured path
STRONG = False
ent_th = 0.95

resnet_res, vgg_res = [], []

for mn, m in u0.items():  # u0 baseline models
    stem = Path(mn).stem

    if STRONG:
        p = ADV_DIR / f"test_apgdce_l2_eps0p7_step0p2_it10_rinit4_from_{stem}.npz"
    else:
        p = ADV_DIR / f"test_apgdce_l2_eps0p5_step0p2_it2_rinit4_from_{stem}.npz"

    X_adv = np.load(p)["x_adv"].astype(np.float32)

    # clean
    clean_pred = m.predict(samples["original_test"], verbose=0)
    clean_lab = np.argmax(clean_pred, axis=1)
    clean_ent = -np.sum(clean_pred * np.log(clean_pred + 1e-12) / np.log(10), axis=1)

    # adv
    adv_pred = m.predict(X_adv, verbose=0)
    adv_lab = np.argmax(adv_pred, axis=1)
    adv_ent = -np.sum(adv_pred * np.log(adv_pred + 1e-12) / np.log(10), axis=1)

    adv_core = adv_ent <= ent_th
    cln_core = clean_ent <= ent_th

    adv_cnt = int(adv_core.sum())
    cln_cnt = int(cln_core.sum())

    adv_cov = adv_cnt / X_adv.shape[0]
    cln_cov = cln_cnt / samples["original_test"].shape[0]

    adv_acc = float(np.mean(adv_lab[adv_core] == truth[adv_core])) if adv_cnt > 0 else 0.0
    cln_acc = float(np.mean(clean_lab[cln_core] == truth[cln_core])) if cln_cnt > 0 else 0.0

    row = [adv_cov, cln_cov, adv_acc, cln_acc]
    (resnet_res if "resnet" in mn else vgg_res).append(row)

print("adv_cov, clean_cov, adv_acc, clean_acc")

print("resnet")
R = np.array(resnet_res, dtype=float)
print(R.mean(axis=0))
print(R.std(axis=0))

print("vgg")
V = np.array(vgg_res, dtype=float)
print(V.mean(axis=0))
print(V.std(axis=0))

In [ ]:
def core_stats(ens, X, y, ent_th, sample_name=""):
    core_preds, core, _ = ens.core_preds(X, sample_name=sample_name, ent_th=float(ent_th))
    cnt = int(core.sum())
    acc = float(np.mean(np.argmax(core_preds, axis=-1) == y[core])) if cnt > 0 else 0.0
    cov = float(core.mean())
    return acc, cov, cnt

truth = y_true["original_test"].reshape(-1)
ent_th = 0.95

U0 = ProbAverageEnsembleFromProbModels(u0)
SPs = ProbAverageEnsembleFromProbModels(sps)
U1 = ProbAverageEnsembleFromProbModels(u1)
cases = [
    ("U0_clean_test",   U0,  samples["original_test"]),
    ("U0_weak_test",    U0,  samples['U0_weak_test']),
    ("U0_strong_test",  U0,  samples['U0_strong_test']),
    ("SPs_weak_test",   SPs, samples['SPs_weak_test']),
    ("SPs_strong_test", SPs, samples['SPs_strong_test']),
    ("U1_weak_test",    U1,  samples['U1_weak_test']),
    ("U1_strong_test",  U1,  samples['U1_strong_test']),
]

for name, ens, X in cases:
    acc, cov, cnt = core_stats(ens, X, truth, ent_th, sample_name=name)
    print(f"{name:10s} | core_acc {acc:.4f} | core_cov {cov:.4f} | core_cnt {cnt}")

## Evaluate Logifold / IMM (if second layer exists)

In [ ]:
TrainData = Tuple[np.ndarray, np.ndarray]
ValData = Tuple[np.ndarray, np.ndarray]

class HistoryDict(TypedDict, total=False):
    history: Dict[str, List[float]]
    params: Dict[str, Any]
    epoch: List[int]


class SpecializeResult(TypedDict):
    model_path: str   
    history: HistoryDict  

        
def specialize(
    new_train : TrainData,
    new_val : ValData,
    original_model : tf.keras.Model,
    new_model_path : str = 'specialized_model.keras',
    path : Path = Path('./data/specialized_models/'),
    verbose = 1,
    epochs = 21
                      ) -> dict:
    """
    Returns (baseline_adv_model, tuned_baseline_adv_model, tuned_history_dict_or_None)
    """
    
    
    x_tr, y_tr = new_train
    x_v, y_v = new_val
    if y_tr.ndim == 1 or y_tr.shape[1] != 10:
        y_tr = to_categorical(y_tr, 10)
    if y_v.ndim == 1 or y_v.shape[1] != 10:
        y_v = to_categorical(y_v, 10)
    if not new_model_path.lower().endswith('.keras'):
        new_model_path += '.keras'
        
    model_path  = path /  Path(new_model_path)    

    if model_path.exists():
        specialized_model = load_model(model_path)
        print(f'{model_path} already exists.')
        hist = custom_specialization.load_history(model_path) 
        
        if hist is None:
            print(f"[WARN] No history found for {model_path}. History is empty dictionary.")
            hist = {}
    else:
        print(f'{model_path} training...')
        specialized_model,hist = custom_specialization.turn_specialist(original_model, path = model_path,
                                                x_tr=x_tr, y_tr=y_tr,
                                                  x_v=x_v,   y_v=y_v,
                                                  epochs=epochs, learning_rate=1e-3, batch_size=128, verbose=verbose, name=f"tuned_once")
        hist = {"history": hist.history, "params": hist.params, "epoch": hist.epoch}
        specialized_model.save(model_path)
    return {
    "model_path": str(model_path),
    "history": hist,
}, specialized_model

In [ ]:
First_layer = u1
baseline = u0
model_dict = u0
specialized_model_dict = sps

In [ ]:
import os, csv
from datetime import datetime
experiment_description = '''
Baseline : Res + VGG
first layer : Res
Second layer : Res
union : clean + weak + strong
'''
def log_metrics_csv(out_csv, row: dict):
    os.makedirs(os.path.dirname(out_csv), exist_ok=True)
    write_header = not os.path.exists(out_csv)
    with open(out_csv, "a", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(row.keys()))
        if write_header:
            w.writeheader()
        w.writerow(row)
        
def log_text(out_txt, text: str):
    os.makedirs(os.path.dirname(out_txt), exist_ok=True)
    with open(out_txt, "a") as f:
        f.write(text + "\n")

In [ ]:
for sample_name, sample in samples.items():
    tokens = sample_name.split('_')
    split = tokens[-1]
#     # union only
    if 'U0' not in sample_name:
        continue
    if split != 'test':
        continue
    else:
        sample_name_head = '_'.join(tokens[:-1])
        sample_val = sample_name_head + '_val'
        sample_train = sample_name_head + '_train'
        sample_test = sample_name_head + '_test'
    
    if sample_val not in samples or sample_train not in samples:
        print(f'[WARN] {sample_name} does not have train/val samples. Skip.')
        continue
    
    print(sample_name, end = ' | ')
    truth = y_true[sample_name]
    
    ent_first = caching.get_entropy(sorted(First_layer.keys()), sample_name)
    
    
    ent_th = round(ent_th_dict[sample_name_head],3)
    print('ent_th:' , ent_th, end = ' | ')
    print('----------------------------------------')
    print('-----------------------------------------------------------------------')
    str_ent_th = str(ent_th).replace('.', 'p')
    out_of_core_th = ent_th


    ## Get Specialized 2nd layer experts
    ## Prepare for OoC samples




    s = np.concatenate([samples[sample_train], samples[sample_val]], axis=0)
    t = np.concatenate([y_true[sample_train], y_true[sample_val]], axis=0)
    new_ent = np.concatenate([caching.get_entropy(sorted(First_layer.keys()), sample_train), caching.get_entropy(sorted(First_layer.keys()), sample_val)], axis = 0)
    OoC_idx = new_ent>out_of_core_th
    OoC_s = s.copy()[OoC_idx]
    OoC_t = t.copy()[OoC_idx]
    if OoC_t.shape[0] < 0.1*t.shape[0]:
            print(f'[WARN] out_of_core_th={out_of_core_th:.3e} -> OoC size too small:', OoC_t.shape[0])
            print('Not preceed further.')
            continue
    OoC_x_train, OoC_x_val, OoC_y_train, OoC_y_val = train_test_split(
        OoC_s, OoC_t,
        test_size=0.1,
        random_state=42,
        stratify=OoC_t
    )
    print(f'For 2nd layer experts, we prepared {OoC_y_train.shape} training samples and {OoC_y_val.shape} validation samples')
    Second_layer={}
    adv_sample_name = sample_name_head + '_OoC_E1E2'+'_'+str_ent_th
    for model_name, m in model_dict.items():
        # ResNet only
        if 'resnet' not in model_name:
            continue
        model_name = model_name[:-6]
        model_name = model_name.split('_')[0] if model_name[0] == 'v' else model_name.split('_')[0] + model_name.split('_')[-1] + 'v'+model_name.split('_')[1][-1]
        model_path = Path('./data/specialized_models/') / Path(f"temp/{model_name}_{adv_sample_name}-SP.keras")
        if model_path.exists():
            Second_layer[f"{model_name}_{adv_sample_name}"] = load_model(model_path)
        else:
            info = specialize(
                    (OoC_x_train, to_categorical(OoC_y_train,10)),
                    (OoC_x_val, to_categorical(OoC_y_val,10)),
                    m,
                    new_model_path=f"temp/{model_name}_{adv_sample_name}-SP",
                verbose = 0)

            Second_layer[f"{model_name}_{adv_sample_name}"] = info[1]

    ## Reload baseline models
    model_dict = {}
    for f_name in sorted(tf_model_dir.glob("*.keras")):
        if f_name.endswith('keras'):
            m = load_model(f"./data/models/{f_name}")
            model_dict[f_name] = m
    First_layer = {}
    First_layer.update(model_dict)
    First_layer.update(specialized_model_dict)

    ## baseline results
    baseline = ProbAverageEnsembleFromProbModels(model_dict)
    p_baseline = baseline.probs(sample, sample_name)
    acc_baseline = np.mean(np.argmax(p_baseline, axis=-1) == truth)

    ## First layer results
    first_layer = ProbAverageEnsembleFromProbModels(First_layer)
    p_first = first_layer.probs(sample, sample_name)
    acc_first = np.mean(np.argmax(p_first, axis=-1) == truth)

    first_core_loc = ent_first<ent_th
    first_core_ent = ent_first[first_core_loc]

    core_p_first = p_first[first_core_loc]
    core_truth = truth[first_core_loc]
    core_acc_first = np.mean(np.argmax(core_p_first, axis=-1) == core_truth)
    OutCore_acc_first = np.mean(np.argmax(p_first[~first_core_loc], axis=-1) == truth[~first_core_loc])

    ## Wrap all_ens and second_layer_ens
    All_layer = {}
    All_layer.update(First_layer)
    All_layer.update(Second_layer)
    all_ens = ProbAverageEnsembleFromProbModels(All_layer)
    second_layer_ens = ProbAverageEnsembleFromProbModels(Second_layer)

    ## All ensemble results
    p_all = all_ens.probs(sample, sample_name)
    acc_all = float(np.mean(np.argmax(p_all, axis=-1) == truth))

    ## Second results
    p_second = second_layer_ens.probs(sample, sample_name)
    acc_second = float(np.mean(np.argmax(p_second, axis=-1) == truth))

    p_second_OutCore = p_second[~first_core_loc]
    acc_second_OutCore = float(np.mean(np.argmax(p_second_OutCore, axis=-1) == truth[~first_core_loc]))

    preds = []
    for model_name in Second_layer.keys():
        pred = caching.get_pred(model_name, sample_name)
        preds.append(pred)
    preds = np.array(preds)
    ent_second = []
    for i in range(preds.shape[1]):
        ent_second.append(utils.cross_entropy(preds[:,i,: ]))
    ent_second = np.array(ent_second)
    ent_second = ent_second[~first_core_loc]
    second_core = ent_second<out_of_core_th
    p_second_core = p_second_OutCore[second_core]
    acc_second_core_of_OutCore = np.mean(np.argmax(p_second_core, axis=-1) == truth[~first_core_loc][second_core])

    ## Gated results (IMM)
    acc_gated = np.sum(np.argmax(p_first[first_core_loc], axis=-1) == truth[first_core_loc]) + np.sum(np.argmax(p_second_OutCore, axis=-1) == truth[~first_core_loc]) 
    acc_gated /= sample.shape[0]

    # See behaviour on AutoAttack samples
    sample_name_aa = 'AA_std_test'
    sample_aa = samples[sample_name_aa]
    truth_aa = y_true[sample_name_aa]

    aa_acc_baseline = float(np.mean(np.argmax(baseline.probs(sample_aa, sample_name_aa), axis=-1) == truth_aa))
    aa_p_first_layer = first_layer.probs(sample_aa, sample_name_aa)
    aa_acc_first = float(np.mean(np.argmax(aa_p_first_layer, axis=-1) == truth_aa))
    aa_first_ent = caching.get_entropy(sorted(First_layer.keys()), sample_name_aa)
    aa_first_core_loc = aa_first_ent<out_of_core_th

    aa_core_p_first = aa_p_first_layer[aa_first_core_loc]
    aa_core_truth = truth_aa[aa_first_core_loc]
    aa_core_acc_first = np.mean(np.argmax(aa_core_p_first, axis=-1) == aa_core_truth)

    aa_p_second = second_layer_ens.probs(sample_aa, sample_name_aa)
    aa_acc_second = float(np.mean(np.argmax(aa_p_second, axis=-1) == truth_aa))

    aa_p_second_OutCore = aa_p_second[~aa_first_core_loc]
    aa_acc_second_OutCore = float(np.mean(np.argmax(aa_p_second_OutCore, axis=-1) == truth_aa[~aa_first_core_loc])) 
    preds = []
    for model_name in Second_layer.keys():
        pred = caching.get_pred(model_name, sample_name_aa)
        preds.append(pred)
    preds = np.array(preds)
    aa_ent_second = []
    for i in range(preds.shape[1]):
        aa_ent_second.append(utils.cross_entropy(preds[:,i,: ]))
    aa_ent_second = np.array(aa_ent_second)
    aa_ent_second = aa_ent_second[~aa_first_core_loc]
    aa_second_core = aa_ent_second<out_of_core_th
    aa_p_second_core = aa_p_second_OutCore[aa_second_core]
    aa_acc_second_core_of_OutCore = np.mean(np.argmax(aa_p_second_core, axis=-1) == truth_aa[~aa_first_core_loc][aa_second_core])

    aa_acc_gated = np.sum(np.argmax(aa_p_first_layer[aa_first_core_loc], axis=-1) == truth_aa[aa_first_core_loc]) + np.sum(np.argmax(aa_p_second_OutCore, axis=-1) == truth_aa[~aa_first_core_loc]) 
    aa_acc_gated /= sample_aa.shape[0]

    aa_acc_all = float(np.mean(np.argmax(all_ens.probs(sample_aa, sample_name_aa), axis=-1) == truth_aa))
    print('----------------------------------------')
    print('----------------------------------------')
    print(f"Sample name : {sample_name} (size : {sample.shape[0]}), out_of_core_th : {out_of_core_th}")
    print(f"First layer acc : {acc_first}")
    print(f"Baseline acc : {acc_baseline}")
    print(f"Second layer acc : {acc_second}")
    print(f"All members ensemble acc : {acc_all}")
    print(f"Entropy threshold for first layer : {out_of_core_th}")
    print(f"First layer core acc : {core_acc_first} (size : {np.sum(first_core_loc)})")
    print(f"First layer OutCore acc : {OutCore_acc_first} (size : {np.sum(~first_core_loc)})")
    print(f"First layer core entropy : {np.sum(first_core_ent)}")
    print(f"First layer entropy : {np.sum(ent_first)}")
    print(f"First layer OutCore entropy : {np.sum(ent_first[~first_core_loc])}")
    print(f"First layer core coverage : {np.sum(first_core_loc) / sample.shape[0]}")
    print(f"Second layer acc on OutCore from first layer  : {acc_second_OutCore} (size : {np.sum(~first_core_loc)})")
    print(f"Second layer core acc on OutCore from first layer : {acc_second_core_of_OutCore} (size : {np.sum(second_core)})")
    print(f"Second layer core entropy on OutCore from first layer : {np.sum(ent_second[second_core])}")
    print(f"Second layer entropy on OutCore from first layer : {np.sum(ent_second)}")
    print(f"Second layer OutCore entropy on OutCore from first layer : {np.sum(ent_second[~second_core])}")
    print(f"Second layer core coverage on OutCore from first layer : {np.sum(second_core) / ent_second.shape[0]}")
    print(f"out core of second layer count : {np.sum(~second_core)}")
    print(f"Gated ensemble acc : {acc_gated}")
    print(f"total entropy = first layer core ent + second layer ent on OutCore from first layer\n{np.sum(first_core_ent)} + {np.sum(ent_second)} = {np.sum(first_core_ent)+np.sum(ent_second)}")
    print("total core entropy (first layer) + core entropy (second layer) : ", np.sum(first_core_ent) + np.sum(ent_second[second_core]))
    print(f"maximum acc among baseline, first layer, second layer : {max(acc_baseline, acc_first, acc_second)}")
    print(f"difference between gated ensemble and maximum acc : {acc_gated - max(acc_baseline, acc_first, acc_second)}")

    print()
    print('AutoAttack result')
    print(f"AA Sample name : {sample_name_aa} (size : {sample_aa.shape[0]}), out_of_core_th : {out_of_core_th}")
    print(f"First layer acc : {aa_acc_first}")
    print(f"Baseline acc : {aa_acc_baseline}")
    print(f"Second layer acc : {aa_acc_second}")
    print(f"All members ensemble acc : {aa_acc_all}")
    print(f"Entropy threshold for first layer : {out_of_core_th}")
    print(f"First layer core acc : {aa_core_acc_first} (size : {np.sum(aa_first_core_loc)})")
    print(f"Second layer acc on OutCore from first layer  : {aa_acc_second_OutCore} (size : {np.sum(~aa_first_core_loc)})")
    print(f"Second layer core acc on OutCore from first layer : {aa_acc_second_core_of_OutCore} (size : {np.sum(aa_second_core)})")
    print(f"Gated ensemble  acc : {aa_acc_gated}")
    print()
    txt = "\n".join([
        "----------------------------------------",
        f"Sample name : {sample_name} (size : {sample.shape[0]}), out_of_core_th : {out_of_core_th}",
        f"First layer acc : {acc_first}",
        f"Baseline acc : {acc_baseline}",
        f"Second layer acc : {acc_second}",
        f"All members ensemble acc : {acc_all}",
        f"Entropy threshold for first layer : {out_of_core_th}",
        f"First layer core acc : {core_acc_first} (size : {np.sum(first_core_loc)})",
        f"First layer OutCore acc : {OutCore_acc_first} (size : {np.sum(~first_core_loc)})",
        f"First layer core entropy : {np.sum(first_core_ent)}",
        f"First layer entropy : {np.sum(ent_first)}",
        f"First layer OutCore entropy : {np.sum(ent_first[~first_core_loc])}",
        f"First layer core coverage : {np.sum(first_core_loc) / sample.shape[0]}",
        f"Second layer acc on OutCore from first layer  : {acc_second_OutCore} (size : {np.sum(~first_core_loc)})",
        f"Second layer core acc on OutCore from first layer : {acc_second_core_of_OutCore} (size : {np.sum(second_core)})",
        f"Second layer core entropy on OutCore from first layer : {np.sum(ent_second[second_core])}",
        f"Second layer entropy on OutCore from first layer : {np.sum(ent_second)}",
        f"out core of second layer count : {np.sum(~second_core)}",
        f"Second layer OutCore entropy on OutCore from first layer : {np.sum(ent_second[~second_core])}",
        f"Second layer core coverage on OutCore from first layer : {np.sum(second_core) / ent_second.shape[0]}",
        f"Gated ensemble acc : {acc_gated}",
        f"total entropy = first layer core ent + second layer ent on OutCore from first layer\n{np.sum(first_core_ent)} + {np.sum(ent_second)} = {np.sum(first_core_ent)+np.sum(ent_second)}",
        f"total core entropy (first layer) + core entropy (second layer) : {np.sum(first_core_ent) + np.sum(ent_second[second_core])}",
        f"maximum acc among baseline, first layer, second layer : {max(acc_baseline, acc_first, acc_second)}",
        f"difference between gated ensemble and maximum acc : {acc_gated - max(acc_baseline, acc_first, acc_second)}",
    ])

    log_text("./analysis/logs/run.txt", txt)
    

    